# PUC Rio | MVP | Machine Learning

Aluno: Joel Carneiro Dutra





## **Descrição do Problema:**

Uma empresa que possui várias franquias de lojas, precisa prever a quantidade de produtos que cada franquia precisará para manter seus estoques otimizados. O objetivo é garantir que cada franquia tenha o estoque adequado para atender à demanda de seus clientes, minimizando ao mesmo tempo os custos de armazenamento e o risco de falta de produtos.

### **Passo 1: Coleta de Dados**

A empresa coleta dados históricos de vendas e estoque de cada uma das franquias, juntamente com informações sobre sazonalidade, promoções, dados geográficos, entre outros fatores que podem influenciar a demanda.

Esta etapa foi realizada por meio de uma extração no Google BigQuery, onde foram selecionadas as dimensões necessárias para o treinamento do modelo. A base possui um histórico de 10 ítens vendidos no ano de 2023, na região Sudeste, o estoque das franquias que efetuaram as vendas. A partir deste ponto, foi realizado o upload da base.

In [ ]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest # para a Seleção Univariada
from sklearn.feature_selection import f_classif # para o teste ANOVA da Seleção Univariada
from sklearn.feature_selection import RFE # para a eliminação recursiva de atributos

from sklearn.linear_model import LogisticRegression # Regressão Logística, para a eliminação recursiva de atributos

from sklearn.model_selection import train_test_split # para particionar em bases de treino e teste (holdout)
from sklearn.model_selection import KFold # para preparar os folds da validação cruzada
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score # para executar a validação cruzada
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score # para a exibição da acurácia do modelo
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier # algoritmo Árvore de Classificação
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC # algoritmo SVM

from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.ensemble import ExtraTreesClassifier # ExtraTrees
from sklearn.ensemble import GradientBoostingClassifier # Gradient Boosting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git init
!git config --global user.name "Joel-Dutra"
!git config --global user.email "dutra.jc@hotmail.com"


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [ ]:
!git add .
!git commit -m "Primeiro commit"
!git remote add origin https://github.com/Joel-Dutra/Joel-Dutra.git
!git push -u origin master


error: open("drive/MyDrive/Anhembi Morumbi/N1 - Comércio Eletrônico.gdoc"): Operation not supported
error: unable to index file 'drive/MyDrive/Anhembi Morumbi/N1 - Comércio Eletrônico.gdoc'
fatal: adding files failed
On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	drive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/Joel-Dutra/Joel-Dutra.git'


In [ ]:
caminho_arquivo = '/content/drive/My Drive/PUC-Rio/base_historica.csv'

In [ ]:
##Leitura do arquivo CSV em um DataFrame
dataset = pd.read_csv(caminho_arquivo)

In [ ]:
dataset.head()

In [ ]:
# Verifica o tipo de dataset de cada atributo
dataset.dtypes

In [ ]:
# Faz um resumo estatístico do dataset (média, desvio padrão, mínimo, máximo e os quartis)
dataset.describe()

In [ ]:
# verificando nulls no dataset
dataset.isnull().sum()

In [ ]:
# distribuição da(s) classe(s)
print(dataset.groupby('estoque_loja').size())

In [ ]:
# salvando um novo dataset para tratamento de missings

# recuperando os nomes das colunas
col = list(dataset.columns)

# o novo dataset irá conter todas as colunas com exceção da última (classe)
atributos = dataset[col[0:-1]]

# substituindo os zeros por NaN
atributos.replace(0, np.nan, inplace=True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

In [ ]:
# removendo as colunas 'qtde_venda', 'qt_dias_sem_estoque' e 'qt_dias_sem_estoque_aberta'
atributos.drop(['qtde_venda', 'qt_dias_sem_estoque', 'qt_dias_sem_estoque_aberta'], axis=1, inplace= True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

In [ ]:
# substituindo os NaN de 'preg' por 0
atributos['qt_dias_loja_fechada'].fillna(0, inplace=True)

# substituindo os NaN de 'plas', 'pres'e 'mass' pela mediana da coluna
atributos['qt_dias_com_estoque'].fillna(atributos['qt_dias_com_estoque'].median(), inplace=True)
atributos['qt_dias_com_estoque_aberta'].fillna(atributos['qt_dias_com_estoque_aberta'].median(), inplace=True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

In [ ]:
# Guardando o novo dataset para testes futuros
datasetSemMissings = atributos

# exibindo as primeiras linhas
datasetSemMissings.head()

In [ ]:
test_size = 0.20
seed = 3

# Separação em conjuntos de treino e teste (dataset original)
array = dataset.values
X = array[:,0:4]
y = array[:,4]
#X_train, X_test, y_train, y_test = train_test_split(X, y,
#    test_size=test_size, shuffle=True, random_state=seed) # sem estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y) # com estratificação

In [ ]:
# Separação em conjuntos de treino e teste (dataset sem missings - 2 colunas a menos!)
array = datasetSemMissings.values
X_sm = array[:,0:2]
y_sm = array[:,2]
#X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm, y_sm,
#    test_size=test_size, shuffle=True, random_state=seed) # sem estratificação
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm, y_sm,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y_sm) # com estratificação

In [ ]:
dataset_atributos = dataset[['qtde_venda','qt_dias_com_estoque','qt_dias_sem_estoque','qt_dias_com_estoque_aberta','qt_dias_sem_estoque_aberta','qt_dias_loja_fechada','estoque_loja']]

In [ ]:
dataset_atributos.head()

In [ ]:
# Histograma
dataset_atributos.hist(figsize = (15,10))
plt.show()

In [ ]:
# Density Plot
dataset_atributos.plot(kind = 'density', subplots = True, layout = (3,3), sharex = False, figsize = (15,10))
plt.show()

In [ ]:
# Matriz de Correlação com Matplotlib Seaborn
sns.heatmap(dataset_atributos.corr(), annot=True, cmap='RdBu');

In [ ]:
# Scatter Plot com Seaborn - Variação 1

sns.pairplot(dataset_atributos)

In [ ]:
# verificando nulls no dataset
dataset_atributos.isnull().sum()

In [ ]:
# Preparação dos dados

# Separação em bases de treino e teste (holdout)
array = dataset_atributos.values
X = array[:,0:6] # atributos
y = array[:,6]   # classe (target)

In [ ]:
# SelectKBest

# Seleção de atributos com SelectKBest
best_var = SelectKBest(score_func=f_classif, k=4)

# Executa a função de pontuação em (X, y) e obtém os atributos selecionados
fit = best_var.fit(X, y)

# Reduz X para os atributos selecionados
features = fit.transform(X)

# Resultados
print('\nNúmero original de atributos:', X.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset_atributos.columns[0:7])

# Exibe as pontuações de cada atributos e os 4 escolhidos (com as pontuações mais altas)
np.set_printoptions(precision=3) # 3 casas decimais
print("\nScores dos Atributos Originais:", fit.scores_)
print("\nAtributos Selecionados:", best_var.get_feature_names_out(input_features=dataset_atributos.columns[0:6]))

In [ ]:
# Eliminação Recursiva de Atributos

# Criação do modelo
modelo = LogisticRegression(max_iter=200)

# Eliminação Recursiva de Atributos
rfe = RFE(modelo, n_features_to_select=3)
fit = rfe.fit(X, y)

# Print dos resultados
print("Atributos Originais:", dataset_atributos.columns[0:7])

# Exibe os atributos selecionados (marcados como True em "Atributos Selecionados"
# e com valor 1 em "Ranking dos Atributos")
print("\nAtributos Selecionados: %s" % fit.support_)
print("\nRanking de atributos: %s" % fit.ranking_)
print("\nQtd de melhores Atributos: %d" % fit.n_features_)
print("\nNomes dos Atributos Selecionados: %s" % fit.get_feature_names_out(input_features=dataset_atributos.columns[0:6]))

In [ ]:
# Importância de Atributos com ExtraTrees

# Criação do modelo para seleção de atributos
modelo = ExtraTreesClassifier(n_estimators=100)
modelo.fit(X,y)

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset_atributos.columns[0:7])

# Exibe a pontuação de importância para cada atributo (quanto maior a pontuação, mais importante é o atributo).
print(modelo.feature_importances_)


SelectKBest:
['qt_dias_com_estoque' 'qt_dias_sem_estoque' 'qt_dias_com_estoque_aberta' 'qt_dias_sem_estoque_aberta']

Logistic Regression:
['qtde_venda' 'qt_dias_sem_estoque' 'qt_dias_sem_estoque_aberta']

ExtraTress:
['qtde_venda' 'qt_dias_com_estoque' 'qt_dias_sem_estoque']

In [ ]:
dataset.dtypes

In [ ]:
dataset.describe()